In [1]:
    import warnings
    warnings.filterwarnings("ignore")
    from sklearn import datasets
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
    import numpy as np
    import tqdm
    plt.style.use('ggplot')
    import lightgbm as ltb

In [2]:
import random

# Set the seed to 42 (you can use any integer value)
random.seed(42)


In [3]:
    import warnings
    warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv("/kaggle/input/valid-dataset-adobe/valid_paths_data.csv")


In [5]:
tem=(pd.to_datetime(data["date"]))

In [6]:
tem1=tem.dt.day_name()

In [7]:

ordinal_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

# One-hot encode with order preservation
def one_hot_encode_ordered(data, ordinal_map):
    encoded_data = pd.DataFrame()
    for day, ordinal in ordinal_map.items():
        column_name = f"Is_{day}"
        encoded_data[column_name] = (data == day).astype(int)
    return encoded_data



In [8]:
encoded_days1 = one_hot_encode_ordered(tem1, ordinal_map)

In [9]:
data = pd.concat([data, encoded_days1], axis=1)

In [10]:
data

,id,date,likes,username,media,inferred company,hour,month,day of week,Year,...,Link,Media Type,image_path,Is_Monday,Is_Tuesday,Is_Wednesday,Is_Thursday,Is_Friday,Is_Saturday,Is_Sunday
0,1,2020-12-12 00:47:00,1,TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,0,12,12,2020,...,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...,0,0,0,0,0,1,0
1,2,2018-06-30 10:04:20,2750,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,10,6,30,2018,...,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...,0,0,0,0,0,1,0
2,3,2020-09-29 19:47:28,57,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,19,9,29,2020,...,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...,0,1,0,0,0,0,0
3,4,2020-10-01 11:40:09,152,MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,11,10,1,2020,...,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...,0,0,0,1,0,0,0
4,5,2018-10-19 14:30:46,41,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,14,10,19,2018,...,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295497,299996,2019-09-07 16:18:10,0,IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,16,9,7,2019,...,https://pbs.twimg.com/media/ED397wUUYAEOV4K?fo...,Photo,/kaggle/input/behavior-challenge-images/media2...,0,0,0,0,0,1,0
295498,299997,2018-02-23 11:24:36,46,CBCOlympics,[Video(thumbnailUrl='https://pbs.twimg.com/med...,cbc,11,2,23,2018,...,https://pbs.twimg.com/media/DWt21i8VMAE69_F.jpg,Video,/kaggle/input/behavior-challenge-images/media2...,0,0,0,0,1,0,0
295499,299998,2020-11-11 20:18:15,261,TeamCoachBuzz,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,20,11,11,2020,...,https://pbs.twimg.com/media/EmkaGpzXIAEyHg8?fo...,Photo,/kaggle/input/behavior-challenge-images/media2...,0,0,1,0,0,0,0
295500,299999,2019-10-29 10:44:00,119,Independent,[Video(thumbnailUrl='https://pbs.twimg.com/med...,independent,10,10,29,2019,...,https://pbs.twimg.com/media/EICgLEQWwAIt3Qr.jpg,Video,/kaggle/input/behavior-challenge-images/media2...,0,1,0,0,0,0,0


In [11]:
text_embedding_path="/kaggle/input/clipx32-batch-mpnetx8-batch/MPNET_Embeds.npy"

In [12]:
image_embedding_path="/kaggle/input/final-image-embeddings-efnet/ENet_Embeds_final.npy"

In [13]:
def load_data(text_embedding_path, image_embedding_path, data):
    text_embeddings = np.load(text_embedding_path)
    image_embeddings = np.load(image_embedding_path)

    
    features = np.array(data[['hour','month', 'Year', ]+list(encoded_days1.columns)])
    
    features = np.concatenate([features,text_embeddings,image_embeddings], axis=1)

    target = np.array(data["likes"])

    return features, target

In [14]:
X,y=load_data(text_embedding_path, image_embedding_path, data)

In [15]:
y=np.log(y+1)

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



# Specify the fraction of the dataset to keep (e.g., 0.1 for 10%)
subsample_fraction = 0.6

# Create bins or quantiles of the target variable
bins = np.linspace(y.min(), y.max(), num=5)  # Adjust the number of bins as needed
y_binned = pd.cut(y, bins=bins, labels=False, include_lowest=True)

# Perform stratified subsampling based on bins
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1 - subsample_fraction, random_state=42, stratify=y_binned
)

In [17]:
X_train.shape

(177301, 1778)

In [18]:
import pandas as pd
import lightgbm as lgb
 
# Similarly LGBMRegressor can also be imported for a regression model.
from lightgbm import LGBMRegressor

In [19]:
my_model = LGBMRegressor(n_estimators=2000,num_leaves=64,lambda_l2=0.1,lambda_l1=0.1,bagging_fraction = 0.8,bagging_freq = 1,min_data_in_leaf=300, learning_rate=0.1, n_jobs=-1,objective='regression', metric='neg_mean_squared_error',verbose=1)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=100, 
             eval_set=(X_test, y_test), 
             eval_metric="mse",verbose=True)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging

LGBMRegressor(bagging_fraction=0.8, bagging_freq=1, lambda_l1=0.1,
              lambda_l2=0.1, metric='neg_mean_squared_error',
              min_data_in_leaf=300, n_estimators=2000, num_leaves=64,
              objective='regression', verbose=1)

In [27]:
from sklearn.metrics import mean_squared_error

predictions = my_model.predict(X_train)
print("Mean squared Error: " + str((mean_squared_error((np.exp(predictions)-1), (np.exp(y_train)-1)))**(0.5)))

Mean squared Error: 3519.878070326631


In [30]:
pd.DataFrame((np.exp(predictions)-1)).describe()

,0
count,177301.000000
mean,527.296397
std,2361.656020
min,-0.710312
25%,3.948913
50%,85.755415
75%,328.773692
max,168150.521284


In [26]:
from sklearn.metrics import mean_squared_error

predictions = my_model.predict(X_train)
print("Mean squared Error: " + str((mean_squared_error(predictions, y_train))**(0.5)))

Mean squared Error: 0.41184959187774717


In [31]:
from sklearn.metrics import mean_squared_error

predictions = my_model.predict(X_test)
print("Mean squared Error: " + str((mean_squared_error((np.exp(predictions)-1), (np.exp(y_test)-1)))**(0.5)))

Mean squared Error: 4519.952921413962


In [32]:
pd.DataFrame((np.exp(predictions)-1)).describe()

,0
count,118201.000000
mean,363.928213
std,2257.095172
min,-0.974174
25%,8.680714
50%,83.170861
75%,264.076558
max,238249.794100


In [13]:
import pickle

In [24]:
with open('lgbm.pkl', 'wb') as f:
    pickle.dump(my_model, f)

In [25]:
with open('lgbm.pkl', 'rb') as f:
    model = pickle.load(f)